In [11]:
from vit_prisma.sae import VisionModelSAERunnerConfig
from vit_prisma.sae import VisionSAETrainer
from vit_prisma.transforms import get_clip_val_transforms


import torchvision
import torch

from torch.utils.data import DataLoader, Subset


# Train the SAE

In [ ]:

# Load an SAE
from huggingface_hub import hf_hub_download, list_repo_files
from vit_prisma.sae import SparseAutoencoder

def load_sae(repo_id, file_name, config_name):
    # Step 1: Download SAE weights and SAE config from Hugginface
    sae_path = hf_hub_download(repo_id, file_name) # Download SAE weights
    hf_hub_download(repo_id, config_name) # Download SAE config

    # Step 2: Now load the pretrained SAE weights from where you just downloaded them
    print(f"Loading SAE from {sae_path}...")
    sae = SparseAutoencoder.load_from_pretrained(sae_path) # This now automatically gets the config.json file in that folder and converts it into a VisionSAERunnerConfig object
    return sae

# Change the repo_id to the Huggingface repo of your chosen SAE. See /docs for a list of SAEs.
repo_id = "Prisma-Multimodal/sparse-autoencoder-clip-b-32-sae-vanilla-x64-layer-10-hook_mlp_out-l1-1e-05" 

file_name = "weights.pt" # Usually weights.pt but may have slight naming variation. See the chosen HF repo for the exact file name
config_name = "config.json"

sae = load_sae(repo_id, file_name, config_name)

Loading SAE from /home/mila/s/sonia.joseph/.cache/huggingface/hub/models--Prisma-Multimodal--sparse-autoencoder-clip-b-32-sae-vanilla-x64-layer-10-hook_mlp_out-l1-1e-05/snapshots/b46d6e9d6c114a53364c5c172ea5023e0e52d4e2/weights.pt...


In [3]:
from vit_prisma.models.model_loader import load_hooked_model

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_name = sae.cfg.model_name
model = load_hooked_model(model_name)
model.to(DEVICE) # Move to device

HookedViT(
  (embed): PatchEmbedding(
    (proj): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32))
  )
  (hook_embed): HookPoint()
  (pos_embed): PosEmbedding()
  (hook_pos_embed): HookPoint()
  (hook_full_embed): HookPoint()
  (ln_pre): LayerNorm(
    (hook_scale): HookPoint()
    (hook_normalized): HookPoint()
  )
  (hook_ln_pre): HookPoint()
  (blocks): ModuleList(
    (0-11): 12 x TransformerBlock(
      (ln1): LayerNorm(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (ln2): LayerNorm(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (attn): Attention(
        (hook_k): HookPoint()
        (hook_q): HookPoint()
        (hook_v): HookPoint()
        (hook_z): HookPoint()
        (hook_attn_scores): HookPoint()
        (hook_pattern): HookPoint()
        (hook_result): HookPoint()
      )
      (mlp): MLP(
        (hook_pre): HookPoint()
        (hook_post): HookPoint()
      )
      (hook_attn_in): Ho

In [14]:
# Put your ImageNet Paths here
from vit_prisma.transforms import get_clip_val_transforms

imagenet_train_path = '/network/scratch/s/sonia.joseph/datasets/kaggle_datasets/ILSVRC/Data/CLS-LOC/train'
imagenet_validation_path = '/network/scratch/s/sonia.joseph/datasets/kaggle_datasets/ILSVRC/Data/CLS-LOC/val'

data_transforms = get_clip_val_transforms()
train_dataset = torchvision.datasets.ImageFolder(imagenet_train_path, transform=data_transforms)
eval_dataset = torchvision.datasets.ImageFolder(imagenet_validation_path, transform=data_transforms)


In [15]:
sae_trainer_cfg = VisionModelSAERunnerConfig(
    hook_point_layer=10,
    layer_subtype='hook_resid_post',
    dataset_name="imagenet",
    feature_sampling_window=1000,
    activation_fn_str='relu',
    checkpoint_path = '/network/scratch/s/sonia.joseph'
)

In [ ]:
trainer = VisionSAETrainer(sae_trainer_cfg, model, train_dataset, eval_dataset)
sae = trainer.run()

